Importing all required dependencies

In [15]:
from itertools import product
from collections import defaultdict
import numpy as np
from scipy.spatial.distance import euclidean
import pulp
import gensim
from sklearn.metrics.pairwise import cosine_similarity

Importing Word2Vec model (https://code.google.com/archive/p/word2vec/). The next block may require a long time!

In [49]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\User\Desktop\projectIR\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

Testing Word2Vec model (@Mario: Nota quanto cazzo è incredibile sta cosa)

In [50]:
x_test = wvmodel['man']+wvmodel['king']-wvmodel['woman']
y_test = wvmodel['queen']
z_test = wvmodel['boat']

In [51]:
print(cosine_similarity([x_test], [y_test]))
print(cosine_similarity([x_test], [z_test]))

[[0.4057604]]
[[0.01503608]]


Here we initialize the collection&nbsp;. https://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html &nbsp;
We use the smaller dataset (called mini). We tokenize all documents and assign to every document an unique ID and a label (depending on the group it came from). 

In [102]:
# To do, parse all documents and get the list of tokens (We remove capital letters?)

Useful functions

In [55]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

Implementation of Word Mover Distance (solved through LP). Next two blocks from: https://github.com/stephenhky/PyWM

In [5]:
def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}

In [70]:
def word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs = {token: wvmodel[token] for token in all_tokens}

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T = pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)

    prob.solve()

    return prob

In [71]:
def WMD_regular(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

We need to define WMD also for a random document with random words

In [94]:
def word_mover_distance_random(documents_token, random_document_vectors, wvmodel, lpFile=None):
    #We create a random association between the random_document_vectors and tokens, assuming random vectors to be all different
    first_sent_tokens = documents_token
    second_sent_tokens = ['__qxvca^&3fd?#_!$' +  str(i) for i in range(len(random_document_vectors))]
    
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs_document = {token: wvmodel[token] for token in first_sent_tokens}
    wordvecs_random = {second_sent_tokens[i]: random_document_vectors[i] for i in range(len(random_document_vectors))}
    #print(wordvecs_random)
    wordvecs = merge_two_dicts(wordvecs_document,wordvecs_random)

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T = pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)

    prob.solve()

    return prob

In [95]:
def WMD_random(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_random(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

Test WMD functions

In [96]:
print(WMD_regular(['hi','bye'],['see','you'],wvmodel))
print(WMD_random(['hi','bye'],[wvmodel['see'],wvmodel['you']],wvmodel))

3.6146241426467896
3.6146241426467896


Implementation of the WME 

In [97]:
def WME_phi(x,w,gamma,wvmodel):
    return np.exp( -gamma*WMD_random(x,w,wvmodel))

In [99]:
def calculate_WME(documents, D_max, R, wvmodel):
    # It must returns a list of text embeddings, i-th element being the embedding of the i-th document
    
    #Phase 1: Compute v_max and v_min
    

SyntaxError: unexpected EOF while parsing (<ipython-input-99-3d0e25626e1d>, line 2)

Dataset import. RECIPE dataset has been used, in json format. The training set contains, for all the recipts, the belonging cuisines.

In [34]:
import json

train_json_data = open('recipe_dataset/train.json')
test_json_data = open('recipe_dataset/test.json')
train_data = json.load(train_json_data)
test_data = json.load(test_json_data)

print(len(train_data))
print(train_data[0]['cuisine'])
print(train_data[0]['ingredients'])

39774
greek
['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']
